This notebook is intended to demonstrate how vessel segmentation methods of ITKTubeTK can be applied to multi-channel MRI (MRA + T1, T2, etc).

In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

In [2]:
ImageType = itk.Image[itk.F, 3]

im1iso = itk.imread("MRA-VesselEnhanced.mha")
im1BrainVess = itk.imread("MRA-Brain-VesselEnhanced.mha")

In [3]:
imMath = ttk.ImageMath.New(im1BrainVess)
imMath.MedianFilter(1)
imMath.Threshold(0.0000000001, 1, 1, 0)
im1VessMask = imMath.GetOutputShort()

ccSeg = ttk.SegmentConnectedComponents.New(im1VessMask)
ccSeg.SetMinimumVolume(4)
ccSeg.Update()
im1VessMaskCC = ccSeg.GetOutput()

In [4]:
imMathSS = ttk.ImageMath.New(im1VessMaskCC)
imMath.Threshold(1,1,0,1)
im1VessMaskInv = imMath.GetOutputFloat()
distFilter = itk.DanielssonDistanceMapImageFilter.New(im1VessMaskInv)
distFilter.Update()
im1SeedRadius = distFilter.GetOutput()

view(im1SeedRadius)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [5]:
numSeeds = 20

vSeg = ttk.SegmentTubes.New(Input=im1iso)
#vSeg.SetVerbose(True)
vSeg.SetMinCurvature(0.0)
vSeg.SetMinRoundness(0.0)
vSeg.SetMinRidgeness(0.8)
vSeg.SetMinLevelness(0.0)
vSeg.SetRadiusInObjectSpace( 0.75 )
vSeg.SetBorderInIndexSpace(3)
vSeg.SetSeedMask( im1SeedRadius )
vSeg.SetSeedRadiusMask( im1SeedRadius )
vSeg.SetOptimizeRadius(False)
vSeg.SetUseSeedMaskAsProbabilities(True)
vSeg.SetSeedExtractionMinimumProbability(0.4)
#vSeg.SetSeedMaskMaximumNumberOfPoints( numSeeds )
vSeg.ProcessSeeds()

In [6]:
tubeMaskImage = vSeg.GetTubeMaskImage()
view(tubeMaskImage)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageF3; pro…

In [8]:
SOWriter = itk.SpatialObjectWriter[3].New(vSeg.GetTubeGroup())
SOWriter.SetFileName( "MRA-Vessels.tre" )
SOWriter.Update()